In [1]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import pathlib

In [1]:
f = open('bind_frequency.json')
data1 = json.load(f)
data2 = dict()
keys = data1.keys()
for k in keys:
    v = data1[k]
    if v in data2:
        data2[v] += 1
    else:
        data2[v] = 1
        
bp = data2.keys()
freq = data2.values()
#print(bp)
#print(freq)
df = pd.DataFrame()
df.reset_index(level=0, inplace=True)
df['partner_count'] = bp
df['frequency'] = freq
df_red = df[df.partner_count > 10]
#print(df_red)
total = df_red.partner_count.sum()
print("Total dropped:", total)
print(df_red.sort_values(by='partner_count'))

Total dropped: 16601
    index  partner_count  frequency
15    NaN             11        223
16    NaN             12        193
31    NaN             13        163
23    NaN             14        147
26    NaN             15        124
..    ...            ...        ...
85    NaN            311          1
74    NaN            379          1
116   NaN            411          1
75    NaN            471          1
88    NaN            630          1

[151 rows x 3 columns]


In [38]:
f = open('bind_frequency.json')
temp = json.load(f)

new_df = pd.DataFrame.from_dict(temp, orient='index')
prot_fr = new_df.stack().reset_index(level=1, drop=True).to_frame(name='freq')
factor = []
prot_fr_r = prot_fr[prot_fr.freq > 10]
for i, row in prot_fr_r.iterrows():
    factor.append(np.round((row['freq']/4.25)))
prot_fr_r['factor'] = factor
prot_fr_r

C:\Users\kmmbd\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,freq,factor
A1L4K1,47,11.0
A5D8V6,32,8.0
A5YKK6,24,6.0
A8K1F4,83,20.0
A8K8P3,20,5.0
...,...,...
O95147,16,4.0
Q9Y570,14,3.0
P0DOE7,25,6.0
Q9BVW5,11,3.0


In [40]:
# save the df
prot_fr_r.to_csv('prot_fr_list_factor.csv', index=True)

In [ ]:
train_file = 'train_positive.csv'
df_train = pd.read_csv(train_file, sep=",", dtype='unicode')
list_A = df_train['int_A']
list_B = df_train['int_B']
list_full = [*list_A, *list_B]
neg_prot = list(prot_fr_r.index.values)

In [90]:
neg_freq_bal = pd.DataFrame(columns=['int_A', 'int_B'])
j = 0
list_A = []
list_B = []
for p in neg_prot:
    f = prot_fr_r.loc[neg_prot[j], 'factor']
    j+=1
    for i in range(int(f)):
        list_A.append(p)
        list_B.append(random.choice(list_full))
#print(list_A)
neg_freq_bal['int_A'] = list_A
neg_freq_bal['int_B'] = list_B
print(neg_freq_bal)

        int_A   int_B
0      A1L4K1  Q81QL7
1      A1L4K1  Q13563
2      A1L4K1  D3JIB2
3      A1L4K1  Q9P2Q2
4      A1L4K1  P38770
...       ...     ...
16808  Q9BVW5  Q9UEW3
16809  Q9BVW5  Q12983
16810  Q96QE5  P02775
16811  Q96QE5  Q9P291
16812  Q96QE5  Q56950

[16813 rows x 2 columns]


In [91]:
neg_freq_bal_r = neg_freq_bal.drop_duplicates()
neg_freq_bal_r

,int_A,int_B
0,A1L4K1,Q81QL7
1,A1L4K1,Q13563
2,A1L4K1,D3JIB2
3,A1L4K1,Q9P2Q2
4,A1L4K1,P38770
...,...,...
16808,Q9BVW5,Q9UEW3
16809,Q9BVW5,Q12983
16810,Q96QE5,P02775
16811,Q96QE5,Q9P291


In [4]:
# Set home directory
home_dir = pathlib.Path.home().joinpath('Documents', 'ms_thesis_ppi')

In [92]:
# check for duplicates in positive set

in_file_2 = pathlib.Path.joinpath(home_dir, 'dataset','9606_Q1.txt')
df_whole = pd.read_csv(in_file_2, sep="\t", header=0, dtype='unicode')
df_whole = df_whole[['UniprotID_A', 'UniprotID_B']] # only keep specific columns
df_whole.columns = ["int_A", "int_B"]
comparison_df = neg_freq_bal_r.merge(df_whole, indicator=True, how='outer')
neg_train_final_balanced = comparison_df.loc[comparison_df['_merge'] == 'left_only']
neg_train_final_balanced.to_csv("neg_train_final_balanced.csv", index=False)
neg_train_final_balanced

,int_A,int_B,_merge
0,A1L4K1,Q81QL7,left_only
1,A1L4K1,Q13563,left_only
2,A1L4K1,D3JIB2,left_only
3,A1L4K1,Q9P2Q2,left_only
4,A1L4K1,P38770,left_only
...,...,...,...
16761,Q9BVW5,Q9UEW3,left_only
16762,Q9BVW5,Q12983,left_only
16763,Q96QE5,P02775,left_only
16764,Q96QE5,Q9P291,left_only


In [114]:
in_file_3 = pathlib.Path.joinpath(home_dir, 'dataset','neg_train_final.csv')
neg_whole = pd.read_csv(in_file_3, sep=",", header=0, dtype='unicode')
neg_frac_1 = neg_whole.sample(frac=0.45)
neg_frac_2 = neg_train_final_balanced.sample(frac=0.2)
neg_ref_bal = pd.concat([neg_frac_1, neg_frac_2])
neg_ref_bal.to_csv('neg_ref_bal.csv', index=False)

In [ ]:
df_reduced = neg_ref_bal[['int_A', 'int_B']].sample(n=21000).reset_index(drop=True)
labels_neg = list(df_reduced.index.values)
# list(map('to_'.__add__,labels))
ID_neg = {}
label_neg = {}
max_label = 1991
for i in range(len(labels_neg)):
    temp_list = [df_reduced.at[i, 'int_A'], df_reduced.at[i, 'int_B']]
    ID_neg[str(max_label+i)] = temp_list
    label_neg[str(max_label+i)] = 0


In [120]:
def save_json(filename, data):
    with open(filename, 'w') as fp:
        json.dump(data, fp)

        
pos_file = pathlib.Path.joinpath(home_dir, 'dataset', 'train', 'train_positive.csv')
df = pd.read_csv(pos_file, sep=",", dtype='unicode')
labels = list(df.index.values)
# list(map('to_'.__add__,labels))
ID = {}
label_pos = {}
max_label = 0
for i in range(len(labels)):
    temp_list = [df.at[i, 'int_A'], df.at[i, 'int_B']]
    ID[str(labels[i])] = temp_list
    label_pos[str(labels[i])] = 1
    max_label = i+1

full_train_set = {**ID, **ID_neg}
#print(sys.getsizeof(full_train_set))
save_json('full_train_set_balanced.json', full_train_set)
    
full_train_label = {**label_pos, **label_neg}
save_json('full_train_label_balanced.json', full_train_label)

In [ ]:
import pathlib 
import random

train_full_imb = open("full_train_set.json")
train_dict = json.load(train_full_imb)
all_keys = list(train_dict.keys())
pos_keys = all_keys[:1990]
neg_keys = all_keys[1990:]
random.shuffle(neg_keys)
#print(neg_keys)
neg_key_b = neg_keys[:2000]
#print(len(neg_key_b))
balanced_train_set = pos_keys + neg_key_b
print(len(balanced_train_set))
train_bal_dict = dict()
for i in balanced_train_set:
    train_bal_dict[i] = train_dict[i]

In [29]:
print(len(train_bal_dict))
# print(train_dict['20684'])
with open('full_train_set_b.json', 'w') as fp:
    json.dump(train_bal_dict, fp)

3990


In [ ]:
#get extra proteins
old_p = pathlib.Path.joinpath(home_dir, 'dataset','train', 'full_train_set.json')
new_p = pathlib.Path.joinpath(home_dir, 'dataset','train', 'full_train_set_balanced.json')
d1 = open(old_p)
d2 = open(new_p)
full = json.load(d1)
bal = json.load(d2)
key_f = full.keys()
key_b = bal.keys()
l_f = []
for k in key_f:
    l_f.append(full[k][0])
    l_f.append(full[k][1])
l_f_filtered = list(set(l_f))
l_b = []
for k in key_b:
    l_b.append(bal[k][0])
    l_b.append(bal[k][1])
l_b_filtered = list(set(l_b))
extra_prot = list(set(l_b_filtered)-set(l_f_filtered))
